
# Model for cascades and tracks

Move to join model for cascade and track events. The cascade model is still for only a single flavour and interaction channel for simplicity.

In [ ]:
import numpy as np
import h5py
from matplotlib import pyplot as plt
import stan_utility
import corner

from joint_model_cascades import * 
from aeff_calculator.aeff_calculator import effective_area_cascades, effective_area_tracks
from energy_conditional import EnergyConditional

In [ ]:
random_seed = 19920715

## Simulation

#### Sources

Assume a predefined set of source and background component with fixed redshift, at the source population redshift threshold. This implements a *very crude* selection effect. 

In [ ]:
# Use a set of sources uniform in comoving volume for a lambda-CDM universe.
with h5py.File('../data/test_SFR_pop.h5', 'r') as f:
    Ns = f['output/Ns'][()]
    redshift = f['output/redshift'][()]
    position = f['output/position'][()]
    
# Select sources within redshift zth to mimic catalogue
zth = 1.0
selection = np.where(redshift < zth)[0]
Ns = len(selection)
redshift = np.array([redshift[_] for _ in selection])
position = np.array([position[_] for _ in selection])
unit_vector = [p / np.linalg.norm(p) for p in position]

# Add background redshift
redshift = list(redshift) + [zth]

#### Effective area
Use same effective area as in precomputation! Now need both the effective area to muon tracks and cascade events/

In [ ]:
from scipy.interpolate import RectBivariateSpline

# Tracks
aeff_tracks = effective_area_tracks()

# Cascades
aeff_cascades = effective_area_cascades("nue_CC")

In [ ]:
# Get input for spline evaluation in Stan
p = 3  # spline degree (default in RectBivariateSpline)

# Tracks
xknots_tracks, yknots_tracks = aeff_tracks.spline.get_knots()
c_tracks = aeff_tracks.spline.get_coeffs()

xknots_tracks = np.unique(xknots_tracks) 
yknots_tracks = np.unique(yknots_tracks)
Nx_tracks = len(xknots_tracks)+p-1
Ny_tracks = len(yknots_tracks)+p-1
c_tracks = c_tracks.reshape(Nx_tracks, Ny_tracks)

# Cascades
xknots_cascades, yknots_cascades = aeff_cascades.spline.get_knots()
c_cascades = aeff_cascades.spline.get_coeffs()

xknots_cascades = np.unique(xknots_cascades)
yknots_cascades = np.unique(yknots_cascades)
Nx_cascades = len(xknots_cascades)+p-1 
Ny_cascades = len(yknots_cascades)+p-1 
c_cascades = c_cascades.reshape(Nx_cascades, Ny_cascades)

# Maximum of the effective area, to scale 
aeff_max_cascades = max(10**aeff_cascades.aeff_vals) # m^2
aeff_max_tracks = np.max(aeff_tracks.aeff_vals) # m^2

#### Precomputed exposure integral
See `precomputation.ipynb` for info.

In [ ]:
# Get precomputed eps factor

# Tracks
with h5py.File('data/precomputed_integral_tracks_1e5.h5', 'r') as f:
    alpha_grid_tracks = f['alpha_grid'][()]
    integral_grid_tracks = np.nan_to_num(f['integral_grid'][()])

# Cascades
with h5py.File('data/precomputed_integral_cascades_6e4.h5', 'r') as f:
    alpha_grid_cascades = f['alpha_grid'][()]
    integral_grid_cascades = f['integral_grid'][()]

In [ ]:
fig, ax = plt.subplots()
for _ in integral_grid_tracks:
    ax.plot(alpha_grid_tracks, _)

#### Energy resolution
See `conditionals.ipynb`.

In [ ]:
# Tracks
# Coming soon

# Cascades
# Nue_CC
cond_file = ('data/conditional_2D_nue_CC_simulation_'
             +'gamma_1.0_lEmin_3.0_lEmax_7.0_fixed.txt')

energy_conditional = EnergyConditional(cond_file)

In [ ]:
# Get input for spline evaluation in Stan
E_p = 1  # spline degree 
E_xknots, E_yknots = energy_conditional.spline.get_knots()
E_xknots = np.unique(E_xknots)
E_yknots = np.unique(E_yknots)
E_Nx = len(E_xknots)+E_p-1 
E_Ny = len(E_yknots)+E_p-1 
E_c = energy_conditional.spline.get_coeffs()
E_c = E_c.reshape(E_Nx, E_Ny)

#### Input parameters

Now define all input parameters, including those that we will fit for.

In [ ]:
# Prepare simulation inputs that are defined above
sim_input = {}

# Sources
sim_input['Ns'] = Ns
sim_input['z'] = redshift
sim_input['D'] = [luminosity_distance(z) for z in redshift[:-1]] # Mpc
sim_input['varpi'] = unit_vector

# Exposure integral interpolation grid
sim_input['Ngrid'] = len(alpha_grid_tracks)
sim_input['alpha_grid_tracks'] = alpha_grid_tracks
sim_input['integral_grid_tracks'] = integral_grid_tracks
sim_input['alpha_grid_cascades'] = alpha_grid_cascades
sim_input['integral_grid_cascades'] = integral_grid_cascades

# Aeff Splines
sim_input['p'] = p

sim_input['Lknots_x_tracks'] = len(xknots_tracks)
sim_input['Lknots_y_tracks'] = len(yknots_tracks)
sim_input['xknots_tracks'] = xknots_tracks
sim_input['yknots_tracks'] = yknots_tracks
sim_input['c_tracks'] = c_tracks
sim_input['aeff_max_tracks'] = aeff_max_tracks

sim_input['Lknots_x_cascades'] = len(xknots_cascades)
sim_input['Lknots_y_cascades'] = len(yknots_cascades)
sim_input['xknots_cascades'] = xknots_cascades
sim_input['yknots_cascades'] = yknots_cascades
sim_input['c_cascades'] = c_cascades
sim_input['aeff_max_cascades'] = aeff_max_cascades

# Eres spline
sim_input['E_p'] = E_p
sim_input['E_Lknots_x'] = len(E_xknots)
sim_input['E_Lknots_y'] = len(E_yknots)
sim_input['E_xknots'] = E_xknots
sim_input['E_yknots'] = E_yknots
sim_input['E_c'] = E_c

In [ ]:
# Define simulation parameters

# Energies
sim_input['alpha'] = 2.0
sim_input['Emin_tracks'] = 1e5 # GeV
sim_input['Emin_cascades'] = 6e4 # GeV

# Reconstruction 
sim_input['kappa_cascades'] = 100 # ~10 deg for cascades
sim_input['kappa_tracks'] = 1000 #~1 degree for tracks

# Associated fraction  (derived paramaters, but easier to define this way)
total_flux = 1e1 # m^-2 yr^-1
f = 0.5

sim_input['T'] = 1 # yr
sim_input['F0'] = (1 - f) * total_flux # m^-2 yr^-1
sim_input['Q'] = ( (f*total_flux) / (sum([1 / (4*np.pi * (d*Mpc_to_m)**2) 
                                          for d in sim_input['D']])) ) # yr^-1 

#### Run simulation

In [ ]:
# Compile Stan model
sim = stan_utility.compile_model(filename = 'stan/tracks_and_cascades_sim.stan', 
                                 model_name = 'tracks_and_cascades_sim', 
                                 include_paths = 'stan/')

In [ ]:
# Simulate
simulation = sim.sampling(data=sim_input, iter=1, chains=1, 
                          algorithm="Fixed_param", seed=random_seed)

In [ ]:
# Extract output
from fancy.interfaces.stan import Direction
from fancy.plotting import AllSkyMap

# Energies
Esrc = simulation.extract(['Esrc'])['Esrc'][0]
E = simulation.extract(['E'])['E'][0]
Edet = simulation.extract(['Edet'])['Edet'][0]

# Arrival directions
event = simulation.extract(['event'])['event'][0]
event = Direction(event)
e_ra = event.d.icrs.ra.rad
e_dec = event.d.icrs.dec.rad

source = Direction(unit_vector)
s_ra = source.d.icrs.ra.rad
s_dec = source.d.icrs.dec.rad

In [ ]:
# Labels
lam = simulation.extract(['lambda'])['lambda'][0] - 1
Nc = Ns
label_cmap = plt.cm.get_cmap('plasma', Nc+1)

lam = list(lam)
Nsrc = sum([lam.count(_) for _ in range(int(Nc))])
Nbg = lam.count(Nc)
print('Nsrc: ', Nsrc)
print('Nbg: ', Nbg)
print('w_bg: ', Nbg / (Nbg+Nsrc))

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

fig, ax = plt.subplots()
fig.set_size_inches((10, 8))
skymap = AllSkyMap(projection = 'hammer', lon_0 = 0, lat_0 = 0);
for r, d, l in zip(e_ra, e_dec, lam):
    color = label_cmap.colors[int(l)]
    skymap.tissot(np.rad2deg(r), np.rad2deg(d), 2, 30, color = color, alpha = 0.7)
norm = max([1 - _/zth for _ in redshift])
for r, d, z in zip(s_ra, s_dec, redshift):
    skymap.tissot(np.rad2deg(r), np.rad2deg(d), 2, 30, color = 'black', 
                  alpha = (1 - z/zth)/norm)

In [ ]:
# Plot energies
Emin = sim_input['Emin']
bins = np.logspace(np.log(Emin/10), np.log(1E+4*Emin), base = np.e)

fig, ax = plt.subplots()
ax.hist(Esrc, bins=bins, label='$\\tilde{E}$', alpha=0.7);
ax.hist(Edet, bins=bins, label='$\hat{E}$', alpha=0.7);
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('$E$ / GeV')
ax.legend();

## Fitting the model

In [ ]:
# For convenience
energy_conditional.get_norm_spline_Edet()

In [ ]:
# Precompute interpolation grids for energy resolution
prob_grid = []
log10_E_grid = []
for lEdet in np.log10(Edet):
    prob_i = []
    log10_E_grid_i = np.linspace(4.0, 7, 100)
    # log10_E_grid_i = np.linspace(lEdet-0.5, lEdet+0.5) # for Nue_CC!
    for lE in log10_E_grid_i:
        prob_i.append(energy_conditional.norm_spline(lE, lEdet)[0][0])
    prob_grid.append(prob_i)
    log10_E_grid.append(log10_E_grid_i)

In [ ]:
# Plot for a random neutrino to show:
fig, ax = plt.subplots()
i=16
ax.scatter(log10_E_grid[i], prob_grid[i])
ax.axvline(np.log10(Edet[i]), label='$E_\mathrm{det}$', color='k', linestyle=':')
ax.set_xlabel('log10($E_\\nu$)', fontsize=14)
ax.set_ylabel('$P(E_\mathrm{det}| E_\\nu)$', fontsize=14)
ax.legend(fontsize=14)

In [ ]:
# Prepare fit inputs
fit_input = {}
fit_input['N'] = len(event.unit_vector)
fit_input['omega_det'] = event.unit_vector
fit_input['Emin'] = sim_input['Emin']
fit_input['Edet'] = Edet
fit_input['Ns'] = Ns
fit_input['varpi'] = source.unit_vector
fit_input['D'] = sim_input['D']
fit_input['z'] = sim_input['z']
fit_input['kappa'] = sim_input['kappa']
fit_input['Ngrid'] = sim_input['Ngrid'] 
fit_input['alpha_grid'] = sim_input['alpha_grid'] 
fit_input['integral_grid'] = sim_input['integral_grid'] 
fit_input['T'] = sim_input['T']

# Spline
fit_input['p'] = sim_input['p']
fit_input['Lknots_x'] = sim_input['Lknots_x'] 
fit_input['Lknots_y'] = sim_input['Lknots_y'] 
fit_input['xknots'] = sim_input['xknots'] 
fit_input['yknots'] = sim_input['yknots'] 
fit_input['c'] = sim_input['c'] 

# Energy interp
fit_input['E_Ngrid'] = len(prob_grid[0])
fit_input['log10_E_grid'] = log10_E_grid
fit_input['prob_grid'] = prob_grid

# Debugging
fit_input['Q_scale'] = sim_input['Q']
fit_input['F0_scale'] = sim_input['F0']

In [ ]:
# Compile the model
model = stan_utility.compile_model(filename = 'stan/joint_cascade_model_Eres.stan', 
                                   model_name = 'Eres_model', 
                                   include_paths = 'stan/')

In [ ]:
fit = model.sampling(data = fit_input, iter = 1000, chains = 4, seed = random_seed)

In [ ]:
# View chains
fig = fit.plot([r'Q', r'F0', r'f', r'alpha', r'F', r'Esrc']);
fig.set_size_inches(20, 16)
fig.tight_layout()

In [ ]:
# View corner plot
chain = fit.extract(permuted = True)
key_list = ['F0', 'Q', 'FT', 'f', 'alpha']
labels = [r'$F_0$', r'Q', r'FT', r'f', 'alpha']
truths = [sim_input['F0'], sim_input['Q'], total_flux, f, sim_input['alpha']]

corner.corner(np.nan_to_num(np.column_stack([chain[key] for key in key_list])),
              labels = labels, 
              truths = truths);

## Association probabilities

In [ ]:
logprob = chain['lp'].transpose(1, 2, 0)
N = np.shape(logprob)[0]

# Account for background component
Ns = np.shape(logprob)[1] - 1

In [ ]:
# Calculate association probabilities for each source-neutrino combo
nu_p = []
for lp in logprob:
    lps = []
    for src in range(Ns+1):
        lps.append(np.mean(np.exp(lp[src])))

    norm = sum(lps)
    ps = []
    for src in range(Ns+1):
        ps.append(lps[src] / norm)
    nu_p.append(ps)

# Normalise line weights
pmax = max(max(nu_p))

In [ ]:
# Figure
fig, ax = plt.subplots()
fig.set_size_inches((12, 6))
skymap = AllSkyMap(projection = 'hammer', lon_0 = 0, lat_0 = 0);

# Sources
for r, d in zip(s_ra, s_dec):
    skymap.tissot(np.rad2deg(r), np.rad2deg(d), 2, 30, color = 'black')
    
# Neutrinos
for r, d, l in zip(e_ra, e_dec, lam):
    color = label_cmap.colors[int(l)]
    skymap.tissot(np.rad2deg(r), np.rad2deg(d), 2, 30, color = color, alpha = 0.7)
    
# Association
for i, p in enumerate(nu_p):
    for j, psrc in enumerate(p[0:Ns]):
        if psrc > 0.001:
            skymap.geodesic(np.rad2deg(e_ra[i]), np.rad2deg(e_dec[i]), 
                            np.rad2deg(s_ra[j]), np.rad2deg(s_dec[j]), 
                            color = 'black', lw = 3, alpha = psrc/pmax, zorder = 10)